In [13]:
import sys
import json
from pathlib import Path
from google.oauth2.credentials import Credentials

root_dir = Path("/workspaces/DeliveryTracker")

sys.path.insert(0, str(root_dir))


credential_json = json.load((root_dir / "google_token/logged_in_credentials.json").open())
credentials = Credentials.from_authorized_user_info(credential_json)

In [14]:
from shared.GoogleServices import GmailService

In [16]:
gs = GmailService(credentials)
gs.authenticate()

In [23]:
from datetime import datetime, timedelta, timezone


time_threshold = datetime.now(timezone.utc) - timedelta(days=7)
query = f"from:no-reply@winsim.de subject:'Ihre winSIM-Rechnung' after:{int(time_threshold.timestamp())}"
    
    # 2. Use the Gmail API to find message IDs that match the query.
response = gs.service.users().messages().list(userId='me', q=query).execute()
messages = response.get('messages', [])

In [24]:
messages

[{'id': '1945b4defe14256d', 'threadId': '1945b4defe14256d'}]

In [26]:
msg = messages[0]

msg_detail = gs.service.users().messages().get(userId='me', id=msg['id'], format='full').execute()

In [28]:
parts = msg_detail['payload'].get('parts', [])
parts

[{'partId': '0',
  'mimeType': 'text/html',
  'filename': '',
  'headers': [{'name': 'Content-Type',
    'value': 'text/html; charset="ISO-8859-1"'},
   {'name': 'Content-Transfer-Encoding', 'value': 'quoted-printable'}],
  'body': {'size': 7927,
   'data': 'PCFET0NUWVBFIGh0bWw-CjxodG1sIGxhbmc9ImVuIiB4bWxuczpvPSJ1cm46c2NoZW1hcy1taWNyb3NvZnQtY29tOm9mZmljZTpvZmZpY2UiIHhtbG5zOnY9InVybjpzY2hlbWFzLW1pY3Jvc29mdC1jb206dm1sIj4KPGhlYWQ-PC9oZWFkPgo8aGVhZD4KICA8bWV0YSBjaGFyc2V0PSJVVEYtOCI-CiAgPG1ldGEgaHR0cC1lcXVpdj0iQ29udGVudC1UeXBlIiBjb250ZW50PSJ0ZXh0L2h0bWw7IGNoYXJzZXQ9dXRmLTgiPgogIDwhLS1baWYgIW1zb10-PCEtLT4KICA8bWV0YSBodHRwLWVxdWl2PSJYLVVBLUNvbXBhdGlibGUiIGNvbnRlbnQ9IklFPWVkZ2UiPgogIDwhLS08IVtlbmRpZl0tLT4KICA8bWV0YSBuYW1lPSJ2aWV3cG9ydCIgY29udGVudD0id2lkdGg9ZGV2aWNlLXdpZHRoLCBpbml0aWFsLXNjYWxlPTEuMCI-CiAgPG1ldGEgbmFtZT0iZm9ybWF0LWRldGVjdGlvbiIgY29udGVudD0idGVsZXBob25lPW5vLCBkYXRlPW5vLCBhZGRyZXNzPW5vLCBlbWFpbD1ubyIgLz4KICA8dGl0bGU-d2luU0lNIFJlY2hudW5nPC90aXRsZT4KPCEtLVtpZiBtc29dPgo8eG1sPgo8bzpPZ

In [30]:
for part in parts:
    filename: str = part['filename']
    if filename.endswith(".pdf"):
        print(filename)
        break

INVB790273815.pdf


In [32]:
attach_id = part['body']['attachmentId']

attachment = gs.service.users().messages().attachments().get(
            userId='me',
            messageId=msg['id'],
            id=attach_id
        ).execute()

In [34]:
import base64
import os


file_data = base64.urlsafe_b64decode(attachment['data'].encode('UTF-8'))
filepath = "testi_attachment.pdf"
with open(filepath, 'wb') as f:
    f.write(file_data)